# Import modules

In [ ]:
import h2o
from h2o.estimators.deepwater import H2ODeepWaterEstimator
import pandas as pd

PATH=os.path.expanduser("~/h2o-3")
h2o.init(port=54321, nthreads=-1)
if not H2ODeepWaterEstimator.available(): exit

# Custom TF LSTM model

In [ ]:
import tensorflow as tf
import json
from keras.layers.core import Dense, Flatten, Reshape
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras import backend as K
from keras.objectives import categorical_crossentropy
from tensorflow.python.framework import ops

def keras_model(n, m):
    # always create a new graph inside ipython or
    # the default one will be used and can lead to
    # unexpected behavior
    graph = tf.Graph() 
    with graph.as_default():
        size = w * h * channels
        # Input images fed via H2O
        inp = tf.placeholder(tf.float32, [None, size])
        # Actual labels used for training fed via H2O
        labels = tf.placeholder(tf.float32, [None, classes])

        # Keras network
        x = Reshape((w, h))(inp)

        x = LSTM(32, input_shape=(w, h))(x)
        x = LSTM(64, input_dim=64, input_length=10, return_sequences=True)(x)
        x = LSTM(32, return_sequences=True)(x)
        x = LSTM(10)(x)
        out = #TODO sum all the neurons into 1 digit

        predictions = out

        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels,logits=out))
        train_step = tf.train.AdamOptimizer(1e-3).minimize(loss)

        init_op = tf.global_variables_initializer()

        # Metadata required by H2O
        tf.add_to_collection(ops.GraphKeys.INIT_OP, init_op.name)
        tf.add_to_collection(ops.GraphKeys.TRAIN_OP, train_step)
        tf.add_to_collection("logits", out)
        tf.add_to_collection("predictions", predictions)

        meta = json.dumps({
                "inputs": {"batch_image_input": inp.name,
                           "categorical_labels": labels.name},
                "outputs": {"categorical_logits": out.name,
                            "layers": ','.join([m.name for m in tf.get_default_graph().get_operations()])},
                "parameters": {}
            })
        tf.add_to_collection("meta", meta)

        # Save the meta file with the graph
        saver = tf.train.Saver()
        filename = "/tmp/keras_tensorflow.meta"
        tf.train.export_meta_graph(filename, saver_def=saver.as_saver_def())

        return filename

# Load and prepare data

In [ ]:
#
# Function which "windows" the input matrix i.e. for matrix:
# 1   2   3
# 4   5   6
# 7   8   9
# 10 11  12
#
# And window 3 the result would be:
# 1  2  3  4  5  6  7   8   9
# 4  5  6  7  8  9  10 11  12
#
# It appends window-1 following rows to each row creating matrix of size dim(rows-window+1, columns*window)
def ngram(inp, window):
    rows = inp.shape[0]
    cols = inp.shape[1]
    resRows = rows - window + 1

    res = DataFrame(columns=range(window*cols))
    for idx in range(resRows):
        if window-1 > 0:
            # TODO REDO IN PYTHON
            newRow = c(inp[idx,], t(inp[(idx+1):(idx+window-1),]))
        elif:
            newRow = inp[idx,]

        if idx % 10000 == 0:
            print(idx)
        res[idx,] = t(newRow)
    return(res)

# Read training data into memory
iotRaw = pd.from_csv('../resources/normal_20170202_2229.csv')

# Select training columns
iot = iotRaw[["LinAccX..g.","LinAccY..g.","LinAccZ..g."]]

# Set training window and ngram
window = 200
iot = ngram(iot, window)

# Send the data to H2O
iot_train  = h2o.H2OFrame(iot)

# TODO - add "y" labels being the sum of all values

# Train the model

In [ ]:
model_filename = keras_model()
model = H2ODeepWaterEstimator(epochs=500, 
                              network_definition_file=model_filename,
                              backend="tensorflow", 
                             ) 
model.train(x = ALL_BUT_LAST, y = LAST_COLUMN, training_frame=iot_train)
model.show()